<h1>Pre-trained models with Resnet-18 - PyTorch</h1>


<h2 id="download_data">Download Data</h2>


In this section, you are going to download the data from IBM object storage using <b>wget</b>, then unzip them.  <b>wget</b> is a command the retrieves content from web servers, in this case its a zip file. Locally we store the data in the directory  <b>/resources/data</b> . The <b>-p</b> creates the entire directory tree up to the given directory.


First, we download the file that contains the images, if you dint do this in your first lab uncomment:


In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip #-P /resources/data

--2021-01-16 10:37:04--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘concrete_crack_images_for_classification.zip.7’

concrete_crack_imag 100%[===================>] 233.90M  40.4MB/s    in 5.8s    

2021-01-16 10:37:10 (40.5 MB/s) - ‘concrete_crack_images_for_classification.zip.7’ saved [245259777/245259777]



We then unzip the file, this ma take a while:


In [2]:
!unzip -q -o concrete_crack_images_for_classification.zip #-d  /resources/data

We then download the files that contain the negative images:


<h2 id="auxiliary">Imports and Auxiliary Functions</h2>


The following are the libraries we are going to use for this lab:


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
!pip install torchvision==0.2.0
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch import optim 

<h2 id="data_class">Dataset Class</h2>


In this section, we will use the previous code to build a dataset class. As before, make sure the even samples are positive, and the odd samples are negative.  If the parameter <code>train</code> is set to <code>True</code>, use the first 30 000  samples as training data; otherwise, the remaining samples will be used as validation data. Do not forget to sort your files so they are in the same order.  


In [4]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory=""
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)    
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        
        
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

<h2 id="trasform_Data_object">Transform Object and Dataset Object</h2>


Create a transform object, that uses the <code>Compose</code> function. First use the transform <code>ToTensor()</code> and followed by <code>Normalize(mean, std)</code>. The value for <code> mean</code> and <code>std</code> are provided for you.


In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

composed = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize(mean, std)])


Create object for the training data  <code>train_datase</code> and validation <code>validation_dataset</code>. Use the transform object to convert the images to tensors using the transform object:


In [6]:
train_dataset=Dataset(transform=composed,train=True)
validation_dataset=Dataset(transform=composed,train=False)

We  can find the shape of the image:


In [7]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

<h2>  Pre-trained models with Resnet-18 <h2>

#### torchvision.models.resnet18(pretrained=False, progress=True, **kwargs)
ResNet-18 model from “Deep Residual Learning for Image Recognition”
##### Parameters
<li>pretrained (bool) – If True, returns a model pre-trained on ImageNet</li>
<li>progress (bool) – If True, displays a progress bar of the download to stderr</li>

<b>Parameter Values</b>

   <li>learning rate:0.1 </li>
   <li>Loss function:Cross Entropy Loss </li>
   <li>epochs:20</li>
   <li>set: torch.manual_seed(0)</li>


In [8]:
torch.manual_seed(0)

learning_rate = 0.05

<b>Model Object:</b>


In [9]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
        param.requeres_grad = False

#model.fc = nn.Linear(512,7)

In [10]:
## ADD LAYER
model.fc = nn.Sequential(
    nn.Linear(512,256),
    nn.BatchNorm1d(256),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

<b>Data Loader Training and Validation:</b>


In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=200)

<b>Criterion:</b>


In [12]:
criterion = nn.CrossEntropyLoss()

<b>Optimizer:</b>


In [13]:
optimizer = torch.optim.Adam ([parameters for parameters in model.parameters()], lr=learning_rate)

<b>Train Model with 20 epochs </b>


In [ ]:
# Train the model

n_epochs = 3
loss_list = []
accuracy_list = []
correct = 0
n_test = len(validation_dataset)

def train_model(n_epochs):
    for epoch in range(n_epochs):
        loss_sublist = []
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss_sublist.append(loss.data.item())
            loss.backward()
            optimizer.step()
        loss_list.append(np.mean(loss_sublist))
        
        correct = 0
        # perform a prediction on the validationdata  
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / n_test
        accuracy_list.append(accuracy)

train_model(n_epochs)

In [ ]:
# Plot the loss and accuracy

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(loss_list,color=color)
ax1.set_xlabel('epoch',color=color)
ax1.set_ylabel('total loss',color=color)
ax1.tick_params(axis='y', color=color)
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color)  
ax2.plot( accuracy_list, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()

In [ ]:
# Question 2
print(accuracy)

plt.plot(loss_list)
plt.xlabel('epoch - itteration')
plt.ylabel('total loss')
plt.plot(show)


In [ ]:
# Question 3


if yhat != y_test & count < max_num_of_items:
    print ("Sample : {}; Expected Label: {}; Obtained Label: {}".format(str(i), str(y_test), str(yhat)))
    count += 1

